# Solver Walkthrough: `assemble_system` & `solve_kkt` Step by Step

This notebook walks through every step of the constrained dynamics solver using a **concrete 2-body example**:
- **Body A** (10 kg) — a box sitting at the origin
- **Body B** (5 kg) — connected to Body A by a **distance constraint** (rigid rod of length 2 m)

We apply **gravity** to both bodies and show, step by step, how `assemble_system` builds the matrices and how `solve_kkt` solves for the constrained accelerations.

---

In [ ]:
import numpy as np
np.set_printoptions(precision=4, suppress=True, linewidth=120)

from aerislab.dynamics.body import RigidBody6DOF
from aerislab.dynamics.constraints import DistanceConstraint
from aerislab.core.solver import assemble_system, solve_kkt

## 1. Create two rigid bodies

We create two simple bodies:
- **Body A**: 10 kg, at position `(0, 0, 0)`, identity orientation
- **Body B**: 5 kg, at position `(2, 0, 0)`, identity orientation

Both use a simple diagonal inertia tensor (like a uniform sphere).

In [ ]:
# Inertia tensors (diagonal = uniform sphere approximation)
I_A = np.diag([1.0, 1.0, 1.0])   # kg·m²
I_B = np.diag([0.5, 0.5, 0.5])   # kg·m²

# Identity quaternion [x, y, z, w] (scalar-last convention)
q_identity = np.array([0.0, 0.0, 0.0, 1.0])

body_A = RigidBody6DOF(
    name="Body_A",
    mass=10.0,                                     # 10 kg
    inertia_tensor_body=I_A,
    position=np.array([0.0, 0.0, 0.0]),            # at origin
    orientation=q_identity,
    linear_velocity=np.array([0.0, 0.0, 0.0]),     # at rest
    angular_velocity=np.array([0.0, 0.0, 0.0]),
)

body_B = RigidBody6DOF(
    name="Body_B",
    mass=5.0,                                      # 5 kg
    inertia_tensor_body=I_B,
    position=np.array([2.0, 0.0, 0.0]),            # 2 m to the right
    orientation=q_identity,
    linear_velocity=np.array([0.0, 0.0, 0.0]),     # at rest
    angular_velocity=np.array([0.0, 0.0, 0.0]),
)

bodies = [body_A, body_B]
print(f"Body A: mass={body_A.mass} kg, position={body_A.p}")
print(f"Body B: mass={body_B.mass} kg, position={body_B.p}")

## 2. Apply gravity

We apply gravity (-9.81 m/s² in the Z direction) as an external force on each body. This is what the solver will see as the "applied forces" `F`.

In [ ]:
g = np.array([0.0, 0.0, -9.81])  # m/s²

for b in bodies:
    b.clear_forces()
    gravity_force = b.mass * g
    b.apply_force(gravity_force)
    print(f"{b.name}: gravity force = {gravity_force} N")

print(f"\nBody A accumulated force: {body_A.f}")
print(f"Body A accumulated torque: {body_A.tau}")
print(f"Body B accumulated force: {body_B.f}")
print(f"Body B accumulated torque: {body_B.tau}")

## 3. Create a distance constraint

We connect Body A and Body B with a **rigid rod** (distance constraint, L = 2 m).  
The attachment points are at the center of mass of each body (local offset = [0,0,0]).

This constraint says: *"The distance between the two attachment points must always be exactly 2 m."*

In [ ]:
constraint = DistanceConstraint(
    world_bodies=bodies,
    body_i=0,                                       # Body A index
    body_j=1,                                       # Body B index
    attach_i_local=np.array([0.0, 0.0, 0.0]),       # attach at center of mass
    attach_j_local=np.array([0.0, 0.0, 0.0]),       # attach at center of mass
    length=2.0,                                     # 2 meters
)

constraints = [constraint]

print(f"Constraint type: DistanceConstraint")
print(f"  Bodies: {constraint.i} (A) ↔ {constraint.j} (B)")
print(f"  Required length: {constraint.L} m")
print(f"  Number of constraint equations: {constraint.rows()}")
print(f"  Current violation C: {constraint.evaluate()}")
print(f"  (C=0 means constraint is satisfied)")

---

# Now let's walk through `assemble_system` step by step

The function signature:
```python
assemble_system(bodies, constraints, alpha=0.0, beta=0.0)
```

We use `alpha=5.0, beta=1.0` for Baumgarte stabilization.

In [ ]:
alpha = 5.0  # position correction strength
beta  = 1.0  # velocity correction strength

nb = len(bodies)       # number of bodies = 2
nv = 6 * nb            # total DOFs = 12  (6 per body: 3 linear + 3 angular)

print(f"Number of bodies (nb): {nb}")
print(f"Total degrees of freedom (nv): {nv}")
print(f"  → Body A uses DOFs  0..5  (indices 0-2: linear, 3-5: angular)")
print(f"  → Body B uses DOFs  6..11 (indices 6-8: linear, 9-11: angular)")

## Step 1: Build the inverse mass matrix `M⁻¹`

Each body contributes a 6×6 block on the diagonal:
```
Wᵢ = [ (1/m) * I₃     0       ]
      [    0        I_world⁻¹  ]
```

This tells us how easily each body accelerates in response to a force.

In [ ]:
Minv = np.zeros((nv, nv), dtype=np.float64)

for i, b in enumerate(bodies):
    Wi = b.inv_mass_matrix_world()
    Minv[6*i:6*i+6, 6*i:6*i+6] = Wi
    
    print(f"--- {b.name} (index {i}) ---")
    print(f"  Inverse mass: 1/m = {b.inv_mass:.4f}  (m = {b.mass} kg)")
    print(f"  6×6 inverse mass block Wᵢ:")
    print(Wi)
    print()

print("\nFull 12×12 inverse mass matrix M⁻¹ (mostly zeros off-diagonal):")
print(Minv)

## Step 2: Compute gyroscopic torque and generalized forces

The gyroscopic torque arises from rotating bodies:
$$\tau_{\text{gyro}} = -\omega \times (I_{\text{world}} \cdot \omega)$$

Since both bodies are at rest (ω = 0), the gyroscopic torque is zero here. But we show the computation anyway.

In [ ]:
F = np.zeros(nv, dtype=np.float64)
v = np.zeros(nv, dtype=np.float64)

for i, b in enumerate(bodies):
    # Gyroscopic torque
    I_world = b.inertia_world()
    tau_gyro = -np.cross(b.w, I_world @ b.w)
    
    # Generalized force = [external force; external torque + gyroscopic torque]
    F_gen = b.generalized_force()
    F_gen[3:6] += tau_gyro
    
    # Store into global vectors
    F[6*i:6*i+6] = F_gen
    v[6*i:6*i+3] = b.v
    v[6*i+3:6*i+6] = b.w
    
    print(f"--- {b.name} (index {i}) ---")
    print(f"  Angular velocity ω:       {b.w}")
    print(f"  I_world @ ω:              {I_world @ b.w}")
    print(f"  Gyroscopic torque:         {tau_gyro}")
    print(f"  External force (gravity):  {b.f}")
    print(f"  External torque:           {b.tau}")
    print(f"  Combined generalized force: {F_gen}")
    print()

print("Global force vector F (12,):")
print(F)
print("\nGlobal velocity vector v (12,):")
print(v)

## Step 3: Assemble the constraint Jacobian `J`

For each constraint, we compute its **local Jacobian** (how the constraint rate-of-change depends on body velocities) and **scatter** it into the correct columns of the global Jacobian.

The distance constraint gives a 1×12 local Jacobian:
$$J_{\text{local}} = [d^T, \; (r_i \times d)^T, \; -d^T, \; -(r_j \times d)^T]$$

where **d** = vector from body B's attachment to body A's attachment.

In [ ]:
# Count total constraint rows
m = sum(c.rows() for c in constraints)
print(f"Total constraint equations (m): {m}")
print(f"  (1 distance constraint → 1 scalar equation)\n")

J = np.zeros((m, nv), dtype=np.float64)
rhs = np.zeros(m, dtype=np.float64)

row = 0
for c in constraints:
    r = c.rows()                 # number of rows this constraint contributes
    i_body, j_body = c.index_map()    # body indices
    Jloc = c.jacobian()          # local Jacobian (r × 12)
    
    print(f"Constraint: {type(c).__name__}")
    print(f"  Connects body {i_body} (A) ↔ body {j_body} (B)")
    print(f"  Rows: {r}")
    print(f"  Local Jacobian Jloc ({r}×12):")
    print(f"    Columns 0-2  (body A linear):  {Jloc[0, 0:3]}")
    print(f"    Columns 3-5  (body A angular):  {Jloc[0, 3:6]}")
    print(f"    Columns 6-8  (body B linear):  {Jloc[0, 6:9]}")
    print(f"    Columns 9-11 (body B angular):  {Jloc[0, 9:12]}")
    print()
    
    # Scatter into global Jacobian
    J[row:row+r, 6*i_body:6*i_body+6] = Jloc[:, 0:6]   # body A columns
    J[row:row+r, 6*j_body:6*j_body+6] = Jloc[:, 6:12]  # body B columns
    
    print(f"  After scattering into global J:")
    print(f"    J[{row}:{row+r}, {6*i_body}:{6*i_body+6}] ← body A part")
    print(f"    J[{row}:{row+r}, {6*j_body}:{6*j_body+6}] ← body B part")
    
    row += r

print(f"\nGlobal constraint Jacobian J ({m}×{nv}):")
print(J)

## Step 4: Compute the constraint velocity `Jv`

The constraint velocity tells us how fast the constraint is currently being violated.

$$\dot{C} = J_{\text{local}} \cdot v_{\text{local}}$$

Since both bodies are at rest, `Jv = 0` here. But the code still computes it.

In [ ]:
# Recompute to show the intermediate values
row = 0
for c in constraints:
    r = c.rows()
    i_body, j_body = c.index_map()
    Jloc = c.jacobian()
    
    # Concatenate velocities of the two connected bodies
    v_loc = np.concatenate([v[6*i_body:6*i_body+6], v[6*j_body:6*j_body+6]])
    Jv = Jloc @ v_loc
    
    print(f"Local velocity vector v_loc (12,):")
    print(f"  Body A: v={v_loc[0:3]}, ω={v_loc[3:6]}")
    print(f"  Body B: v={v_loc[6:9]}, ω={v_loc[9:12]}")
    print(f"\nConstraint velocity Jv = Jloc @ v_loc = {Jv}")
    print(f"  (zero because both bodies are stationary)")
    
    row += r

## Step 5: Evaluate constraint violation `C`

The distance constraint measures:
$$C = \frac{1}{2}(\|d\|^2 - L^2)$$

If C = 0, the constraint is perfectly satisfied (bodies are exactly L apart).

In [ ]:
for c in constraints:
    C = c.evaluate()
    
    # Let's also manually verify:
    d = body_A.p - body_B.p   # vector from B to A
    dist = np.linalg.norm(d)
    
    print(f"Vector d = pA - pB = {d}")
    print(f"Current distance ||d||: {dist:.4f} m")
    print(f"Required distance L:    {c.L:.4f} m")
    print(f"Constraint violation C = 0.5*(||d||² - L²) = {C}")
    print(f"  → C = 0 means constraint is perfectly satisfied ✓")

## Step 6: Compute Baumgarte-stabilized RHS

$$\text{rhs} = -(1 + \beta) \cdot Jv - \alpha \cdot C$$

This is the "target" for the constraint solver:
- **`-(1+β)·Jv`** acts like a **damper** — pushes the constraint velocity toward zero
- **`-α·C`** acts like a **spring** — pushes the constraint violation back toward zero

In [ ]:
rhs = np.zeros(m, dtype=np.float64)

row = 0
for c in constraints:
    r = c.rows()
    i_body, j_body = c.index_map()
    Jloc = c.jacobian()
    
    v_loc = np.concatenate([v[6*i_body:6*i_body+6], v[6*j_body:6*j_body+6]])
    Jv = Jloc @ v_loc
    C = c.evaluate()
    
    rhs[row:row+r] = -(1.0 + beta) * Jv - alpha * C
    
    print(f"Baumgarte stabilization:")
    print(f"  α = {alpha} (position correction strength)")
    print(f"  β = {beta} (velocity correction strength)")
    print(f"  Jv = {Jv}")
    print(f"  C  = {C}")
    print(f"  -(1+β)·Jv = {-(1.0 + beta) * Jv}")
    print(f"  -α·C      = {-alpha * C}")
    print(f"  rhs = -(1+β)·Jv - α·C = {rhs[row:row+r]}")
    
    row += r

print(f"\nFinal RHS vector: {rhs}")

## Step 7: Verify against `assemble_system` function

Let's call the actual function and check that our manual computation matches.

In [ ]:
# Call the actual function
Minv_actual, J_actual, F_actual, rhs_actual, v_actual = assemble_system(
    bodies, constraints, alpha=alpha, beta=beta
)

print("Do our manual results match the function output?")
print(f"  M⁻¹ matches: {np.allclose(Minv, Minv_actual)}")
print(f"  J   matches: {np.allclose(J, J_actual)}")
print(f"  F   matches: {np.allclose(F, F_actual)}")
print(f"  rhs matches: {np.allclose(rhs, rhs_actual)}")
print(f"  v   matches: {np.allclose(v, v_actual)}")
print("\n✓ All matched!")

---

# Now let's walk through `solve_kkt` step by step

The KKT system solves:
$$M \cdot a = F + J^T \cdot \lambda$$
$$J \cdot a = \text{rhs}$$

Using the Schur complement method:
1. Compute unconstrained acceleration: $a_0 = M^{-1} \cdot F$
2. Form effective mass matrix: $A = J \cdot M^{-1} \cdot J^T$
3. Solve for multipliers: $A \cdot \lambda = \text{rhs} - J \cdot a_0$
4. Compute constrained acceleration: $a = a_0 + M^{-1} \cdot J^T \cdot \lambda$

### KKT Step 1: Unconstrained acceleration `a₀ = M⁻¹ · F`

This is what the accelerations would be if there were **no constraints** — just free bodies under gravity.

In [ ]:
a0 = Minv @ F

print("Unconstrained accelerations a₀ = M⁻¹ · F:")
print(f"  Body A linear  (a₀[0:3]):  {a0[0:3]}  m/s²")
print(f"  Body A angular (a₀[3:6]):  {a0[3:6]}  rad/s²")
print(f"  Body B linear  (a₀[6:9]):  {a0[6:9]}  m/s²")
print(f"  Body B angular (a₀[9:12]): {a0[9:12]} rad/s²")
print(f"\n→ Both bodies would accelerate at -9.81 m/s² in Z (free fall).")
print(f"  No angular acceleration because torque = 0.")

### KKT Step 2: Effective mass matrix `A = J · M⁻¹ · Jᵀ`

This is the "constraint mass" — how much resistance the system has against constraint-direction acceleration.

For our 1 constraint, A is a 1×1 matrix (a scalar).

In [ ]:
A = J @ (Minv @ J.T)

print(f"Effective constraint mass matrix A = J · M⁻¹ · Jᵀ:")
print(f"  Shape: {A.shape}  (1 constraint → 1×1 scalar)")
print(f"  A = {A}")
print(f"\n  This represents the 'effective inverse mass' along the constraint direction.")
print(f"  Breakdown: 1/m_A · ||d||² + 1/m_B · ||d||²")
d = body_A.p - body_B.p
manual_A = (1.0/body_A.mass + 1.0/body_B.mass) * np.dot(d, d)
print(f"  = (1/{body_A.mass} + 1/{body_B.mass}) × ||d||² = {manual_A:.4f}")

### KKT Step 3: Solve for Lagrange multiplier `λ`

$$A \cdot \lambda = \text{rhs} - J \cdot a_0$$

The Lagrange multiplier `λ` tells us the **magnitude of the constraint force** needed to keep the constraint satisfied.

In [ ]:
b_rhs = rhs - J @ a0

print(f"RHS for lambda equation:")
print(f"  rhs     = {rhs}")
print(f"  J · a₀  = {J @ a0}")
print(f"  b = rhs - J·a₀ = {b_rhs}")
print()

lam = np.linalg.solve(A, b_rhs)
print(f"Lagrange multiplier λ = A⁻¹ · b = {lam}")
print(f"\n→ This is the 'constraint force intensity' along the rod direction.")

### KKT Step 4: Compute constrained acceleration `a = a₀ + M⁻¹ · Jᵀ · λ`

The constraint force modifies the free-fall acceleration so that both bodies respect the rigid rod.

In [ ]:
# Constraint force in generalized coordinates
F_constraint = J.T @ lam

print(f"Constraint force Jᵀ·λ (generalized):")
print(f"  Body A force:  {F_constraint[0:3]} N")
print(f"  Body A torque: {F_constraint[3:6]} N·m")
print(f"  Body B force:  {F_constraint[6:9]} N")
print(f"  Body B torque: {F_constraint[9:12]} N·m")
print()

# Acceleration correction
a_correction = Minv @ F_constraint
print(f"Acceleration correction M⁻¹·Jᵀ·λ:")
print(f"  Body A linear:  {a_correction[0:3]} m/s²")
print(f"  Body B linear:  {a_correction[6:9]} m/s²")

# Final constrained acceleration
a = a0 + a_correction
print(f"\nFinal constrained acceleration a = a₀ + correction:")
print(f"  Body A linear:  {a[0:3]} m/s²")
print(f"  Body A angular: {a[3:6]} rad/s²")
print(f"  Body B linear:  {a[6:9]} m/s²")
print(f"  Body B angular: {a[9:12]} rad/s²")

### Verify against `solve_kkt` function

In [ ]:
a_actual, lam_actual = solve_kkt(Minv, J, F, rhs)

print(f"Does our manual KKT solve match the function?")
print(f"  a   matches: {np.allclose(a, a_actual)}")
print(f"  λ   matches: {np.allclose(lam, lam_actual)}")
print(f"\n✓ All matched!")

---

# Physical interpretation

Let's interpret what the solver computed:

In [ ]:
print("=" * 65)
print("PHYSICAL INTERPRETATION")
print("=" * 65)
print()
print("Setup: Two bodies connected by a horizontal rigid rod (2 m).")
print("Both pulled downward by gravity.")
print()
print(f"Without constraint:")
print(f"  Both bodies: a_z = -9.81 m/s² (free fall)")
print(f"  The rod would be fine — both fall equally.")
print()
print(f"With constraint:")
print(f"  Body A: a = {a[0:3]} m/s²")
print(f"  Body B: a = {a[6:9]} m/s²")
print()
print(f"  → Both still accelerate at -9.81 m/s² in Z.")
print(f"  → No constraint force needed in this configuration!")
print(f"  → Gravity pulls both bodies identically, so the rod")
print(f"    stays at the same length without any tension.")
print()
print(f"  Lagrange multiplier λ = {lam}")
print(f"  (λ ≈ 0 confirms no constraint force is needed)")

---

# Bonus: A more interesting case — bodies at different heights

Let's put Body B **below** Body A so the rod is vertical. Now gravity pulls Body B down but the rod must hold it at a fixed distance. The constraint force (tension in the rod) will be nonzero!

In [ ]:
# Reset bodies: vertical rod configuration
body_A.p[:] = [0.0, 0.0, 2.0]    # A is 2 m above ground
body_B.p[:] = [0.0, 0.0, 0.0]    # B is at ground level
body_A.v[:] = 0.0
body_A.w[:] = 0.0
body_B.v[:] = 0.0
body_B.w[:] = 0.0

# Re-apply gravity
for b in bodies:
    b.clear_forces()
    b.apply_force(b.mass * g)

# Assemble and solve
Minv2, J2, F2, rhs2, v2 = assemble_system(bodies, constraints, alpha=alpha, beta=beta)
a2, lam2 = solve_kkt(Minv2, J2, F2, rhs2)

print("Setup: Vertical rod  (A on top, B on bottom)")
print(f"  Body A position: {body_A.p}  (top)")
print(f"  Body B position: {body_B.p}  (bottom)")
print()
print(f"Constraint Jacobian J:")
print(J2)
print(f"\n  → The constraint direction is along Z (the rod is vertical)")
print()
print(f"Unconstrained accelerations:")
a0_2 = Minv2 @ F2
print(f"  Body A: a_z = {a0_2[2]:.4f} m/s²")
print(f"  Body B: a_z = {a0_2[8]:.4f} m/s²")
print(f"  → Same free-fall for both")
print()
print(f"Constrained accelerations:")
print(f"  Body A: a = {a2[0:3]}  m/s²")
print(f"  Body B: a = {a2[6:9]}  m/s²")
print(f"  → Both still in free fall (rigid body falls as one unit)")
print()
print(f"Lagrange multiplier λ = {lam2}")
print(f"Constraint force on A: {J2.T @ lam2}")

## Now make it interesting: fix Body A (simulate a hanging mass)

If we manually set Body A's acceleration to zero (like it's pinned to the ceiling), the constraint force must support Body B against gravity. Let's see what happens when only body B has gravity.

In [ ]:
# Make body A very heavy (quasi-static, like attached to ceiling)
body_A_heavy = RigidBody6DOF(
    name="Ceiling_Anchor",
    mass=1e6,                                        # "infinitely" heavy
    inertia_tensor_body=np.diag([1e6, 1e6, 1e6]),
    position=np.array([0.0, 0.0, 2.0]),
    orientation=q_identity,
)

body_B_hanging = RigidBody6DOF(
    name="Hanging_Mass",
    mass=5.0,
    inertia_tensor_body=I_B,
    position=np.array([0.0, 0.0, 0.0]),
    orientation=q_identity,
)

bodies_hang = [body_A_heavy, body_B_hanging]

# Only apply gravity to the hanging mass
body_A_heavy.clear_forces()
body_B_hanging.clear_forces()
body_B_hanging.apply_force(body_B_hanging.mass * g)  # only B feels gravity

# Constraint: L = 2 m
constraint_hang = DistanceConstraint(
    world_bodies=bodies_hang,
    body_i=0, body_j=1,
    attach_i_local=np.zeros(3),
    attach_j_local=np.zeros(3),
    length=2.0,
)

Minv_h, J_h, F_h, rhs_h, v_h = assemble_system(
    bodies_hang, [constraint_hang], alpha=alpha, beta=beta
)
a_h, lam_h = solve_kkt(Minv_h, J_h, F_h, rhs_h)

print("HANGING MASS SCENARIO")
print("=" * 50)
print(f"Ceiling anchor (body 0): mass = {body_A_heavy.mass} kg")
print(f"Hanging mass   (body 1): mass = {body_B_hanging.mass} kg")
print()
print(f"Unconstrained acceleration:")
a0_h = Minv_h @ F_h
print(f"  Anchor:  a_z = {a0_h[2]:.6f} m/s²  (≈ 0, very heavy)")
print(f"  Hanging: a_z = {a0_h[8]:.4f} m/s²  (free fall)")
print()
print(f"Constrained acceleration:")
print(f"  Anchor:  a = {a_h[0:3]}")
print(f"  Hanging: a = {a_h[6:9]}")
print(f"  → The hanging mass barely accelerates (held by the rod)!")
print()
print(f"Lagrange multiplier λ = {lam_h}")
F_constraint_hang = J_h.T @ lam_h
print(f"Constraint force on hanging mass: {F_constraint_hang[6:9]} N")
print(f"  → points upward (+Z), opposing gravity")
print(f"  Expected: m·g = {body_B_hanging.mass * 9.81:.2f} N upward")
print(f"  The rod tension almost exactly supports the hanging weight! ✓")

---

# Summary of the solver pipeline

```
┌─────────────────────────┐
│   assemble_system()     │
│                         │
│  For each body:         │
│   ├─ Inverse mass M⁻¹  │
│   ├─ Gyroscopic torque  │
│   ├─ Generalized force  │
│   └─ Current velocity   │
│                         │
│  For each constraint:   │
│   ├─ Local Jacobian     │
│   ├─ Scatter → global J │
│   ├─ Constraint vel Jv  │
│   ├─ Violation C        │
│   └─ Baumgarte RHS     │
└────────────┬────────────┘
             │
             ▼
┌─────────────────────────┐
│     solve_kkt()         │
│                         │
│  1. a₀ = M⁻¹ · F       │
│  2. A  = J·M⁻¹·Jᵀ      │
│  3. λ  = A⁻¹(rhs-J·a₀) │
│  4. a  = a₀ + M⁻¹·Jᵀ·λ │
└────────────┬────────────┘
             │
             ▼
       Constrained
      accelerations
```